# Baichuan2-7B data2text LoRA微调

In [1]:
import os
os.environ["http_proxy"] = "http://10.6.0.17:8888"
os.environ["https_proxy"] = "http://10.6.0.17:8888"

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [9]:
import torch
from transformers import (
    AutoConfig,
    AutoTokenizer,
    AutoModelForCausalLM,
    GenerationConfig
)
from peft import PeftModel
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model_path = 'baichuan-inc/Baichuan2-7B-Chat'
model_path = '/root/.cache/huggingface/hub/models--baichuan-inc--BaiChuan2-13B-Chat/snapshots/c8d877c7ca596d9aeff429d43bff06e288684f45'
# lora_path = 'zjunlp/baichuan2-7b-iepile-lora'
lora_path = '/root/.cache/huggingface/hub/models--zjunlp--baichuan2-13b-iepile-lora/snapshots/8c86af8b52cfc9e2f59ac6f58094263cd20ad28d'
config = AutoConfig.from_pretrained(model_path, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

In [10]:
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    config=config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)
model = PeftModel.from_pretrained(
    model,
    lora_path,
)

model.eval()

Loading checkpoint shards: 100%|██████████| 3/3 [01:00<00:00, 20.25s/it]


Transformers cache directory: /root/.cache/huggingface/hub
/root/.cache/huggingface/hub/models--baichuan-inc--BaiChuan2-13B-Chat/snapshots/c8d877c7ca596d9aeff429d43bff06e288684f45


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): BaichuanForCausalLM(
      (model): BaichuanModel(
        (embed_tokens): Embedding(125696, 5120, padding_idx=0)
        (layers): ModuleList(
          (0-39): 40 x BaichuanLayer(
            (self_attn): BaichuanAttention(
              (W_pack): Linear(
                in_features=5120, out_features=15360, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=15360, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (o_proj): Linear(
                in_features=5120, out_features=5120, bias=Fals

In [2]:
!CUDA_VISIBLE_DEVICES=0 python src/finetune.py \
    --do_train --do_eval \
    --overwrite_output_dir \
    --model_name_or_path 'baichuan-inc/Baichuan2-7B-Chat' \
    --stage 'sft' \
    --model_name 'baichuan' \
    --template 'baichuan2' \
    --train_file 'data2text/data2text-ner-train-transformed.json' \
    --valid_file 'data2text/data2text-ner-val-transformed.json' \
    --output_dir='lora/baichuan7B-test' \
    --per_device_train_batch_size 2 \
    --per_device_eval_batch_size 2 \
    --gradient_accumulation_steps 4 \
    --preprocessing_num_workers 16 \
    --num_train_epochs 10 \
    --learning_rate 5e-5 \
    --max_grad_norm 0.5 \
    --optim "adamw_torch" \
    --max_source_length 400 \
    --cutoff_len 700 \
    --max_target_length 300 \
    --evaluation_strategy "epoch" \
    --save_strategy "epoch" \
    --save_total_limit 10 \
    --lora_r 64 \
    --lora_alpha 64 \
    --lora_dropout 0.05 \
    --bf16 \
    --bits 4


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda117.so
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following direct

### 使用微调模型对测试集进行抽取

In [1]:
!CUDA_VISIBLE_DEVICES=0 python src/inference.py \
    --stage sft \
    --model_name_or_path 'baichuan-inc/Baichuan2-7B-Chat' \
    --checkpoint_dir 'lora/baichuan7B-data2text-continue' \
    --model_name 'baichuan' \
    --template 'baichuan2' \
    --do_predict \
    --input_file 'data2text/data2text-ner-test-transformed.json' \
    --output_file 'results/baichuan7B-ft-v2-test-output.json' \
    --finetuning_type lora \
    --output_dir 'lora/oneke-continue-test' \
    --predict_with_generate \
    --cutoff_len 512 \
    --bf16 \
    --max_new_tokens 300 \
    --bits 4


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda117.so
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib64'), PosixPath('/usr/local/nvidia/lib')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following direct

### 对测试集进行评估

In [17]:
!python ie2instruction/eval_func.py \
  --path1 results/baichuan7B-ft-test-output.json \
  --task NER 

{'total': {'总样本数': 215, '错误数': 0, 'P': 45.35, 'R': 45.8, 'F1': 45.57}}
